In [6]:
import pandas as pd
import numpy as np
import snowflake.connector


In [7]:
conn = snowflake.connector.connect(
    user='**************',
    password='************',
    account='********-********',
    warehouse='COMPUTE_WH',
    database='SALES',
    schema='OBJECTS'
)

cur = conn.cursor()
print("✅ Connected to Snowflake successfully!")

✅ Connected to Snowflake successfully!


In [8]:
stg_df = pd.read_sql("SELECT * FROM SALESRECORD LIMIT 10;", conn)
final_df = pd.read_sql("SELECT * FROM SALESRECORDFINAL LIMIT 10;", conn)

display(stg_df)
display(final_df)

C:\Users\anusr\AppData\Local\Temp\ipykernel_14668\1057023525.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stg_df = pd.read_sql("SELECT * FROM SALESRECORD LIMIT 10;", conn)
C:\Users\anusr\AppData\Local\Temp\ipykernel_14668\1057023525.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  final_df = pd.read_sql("SELECT * FROM SALESRECORDFINAL LIMIT 10;", conn)


,ORDER_DATE,DATETIME,CASH_TYPE,MONEY,COFFEE_NAME
0,2024-02-08,2024-02-08 14:26:04,cash,15.0,Tea
1,2024-02-08,2024-02-08 14:28:26,cash,15.0,Tea
2,2024-02-08,2024-02-08 14:33:04,card,20.0,Espresso
3,2024-02-08,2024-02-08 15:51:04,card,30.0,Chocolate with coffee
4,2024-02-08,2024-02-08 16:35:01,cash,27.0,Chocolate with milk
5,2024-02-08,2024-02-08 18:55:30,card,33.0,Espresso
6,2024-02-08,2024-02-08 18:59:03,cash,29.0,Coffee with Irish whiskey
7,2024-02-08,2024-02-08 19:00:28,cash,27.0,Irish whiskey with milk
8,2024-02-09,2024-02-09 08:45:45,card,28.0,Double Irish whiskey
9,2024-02-09,2024-02-09 10:41:36,cash,30.0,Chocolate with coffee


,ORDER_DATE,DATETIME,CASH_TYPE,MONEY,COFFEE_NAME,TOTAL
0,2024-02-08,2024-02-08 14:26:04,cash,15.0,Tea,15.75
1,2024-02-08,2024-02-08 14:28:26,cash,15.0,Tea,15.75
2,2024-02-08,2024-02-08 14:33:04,card,20.0,Espresso,21.00
3,2024-02-08,2024-02-08 15:51:04,card,30.0,Chocolate with coffee,31.50
4,2024-02-08,2024-02-08 16:35:01,cash,27.0,Chocolate with milk,28.35
5,2024-02-08,2024-02-08 18:55:30,card,33.0,Espresso,34.65
6,2024-02-08,2024-02-08 18:59:03,cash,29.0,Coffee with Irish whiskey,30.45
7,2024-02-08,2024-02-08 19:00:28,cash,27.0,Irish whiskey with milk,28.35
8,2024-02-09,2024-02-09 08:45:45,card,28.0,Double Irish whiskey,29.40
9,2024-02-09,2024-02-09 10:41:36,cash,30.0,Chocolate with coffee,31.50


In [9]:
import pandas as pd
import numpy as np

# ============================================================================
# DATA VALIDATION: Compare Staging vs Final Tables
# ============================================================================

# Load data from database
stg_df = pd.read_sql("SELECT * FROM SALESRECORD;", conn)
final_df = pd.read_sql("SELECT * FROM SALESRECORDFINAL;", conn)

print("="*60)
print("DATA VALIDATION REPORT")
print("="*60 + "\n")

# ----------------------------------------------------------------------------
# TEST 1: Row Count - Do both tables have the same number of rows?
# ----------------------------------------------------------------------------
print("TEST 1: Row Count")
stg_count = len(stg_df)
final_count = len(final_df)
print(f"  Staging: {stg_count:,} rows")
print(f"  Final:   {final_count:,} rows")

if stg_count == final_count:
    print(f"  Result:  PASS\n")
else:
    print(f"  Result:  FAIL\n")

# ----------------------------------------------------------------------------
# TEST 2: Null Values - Are required columns populated?
# ----------------------------------------------------------------------------
print("TEST 2: Null Value Check")

# Check staging table
print("  Staging Table:")
for col in ['ORDER_DATE', 'DATETIME', 'MONEY', 'CASH_TYPE', 'COFFEE_NAME']:
    if col in stg_df.columns:
        nulls = stg_df[col].isnull().sum()
        status = "PASS" if nulls == 0 else "FAIL"
        print(f"    {col}: {nulls} nulls [{status}]")

# Check final table
print("\n  Final Table:")
for col in ['ORDER_DATE', 'DATETIME', 'MONEY', 'CASH_TYPE', 'COFFEE_NAME', 'TOTAL']:
    if col in final_df.columns:
        nulls = final_df[col].isnull().sum()
        status = "PASS" if nulls == 0 else "FAIL"
        print(f"    {col}: {nulls} nulls [{status}]")

print()

# ----------------------------------------------------------------------------
# TEST 3: Business Logic - Is TOTAL_AMOUNT calculated correctly?
# Expected: TOTAL = MONEY * 1.05 (5% markup)
# ----------------------------------------------------------------------------
print("TEST 3: Business Logic Validation")
print("  Formula: TOTAL = MONEY * 1.05")

if 'TOTAL' in final_df.columns and 'MONEY' in final_df.columns:
    # Calculate what TOTAL_AMOUNT should be
    expected = final_df['MONEY'] * 1.05
    actual = final_df['TOTAL']
    
    # Compare (using np.isclose to handle decimal precision)
    correct = np.isclose(actual, expected, rtol=1e-09, atol=1e-09)
    errors = (~correct).sum()
    
    print(f"  Errors: {errors}")
    
    if errors == 0:
        print(f"  Result: PASS")
    else:
        print(f"  Result: FAIL")
        
        # Show examples if there are errors
        print("\n  Sample errors:")
        error_rows = final_df[~correct][['MONEY', 'TOTAL']].head(3).copy()
        error_rows['Expected'] = expected[~correct].head(3).values
        print(error_rows.to_string(index=False))
else:
    print("  WARNING: Required columns not found")

print()

# ----------------------------------------------------------------------------
# TEST 4: Data Consistency - Do common columns match between tables?
# ----------------------------------------------------------------------------
print("TEST 4: Data Consistency Check")

# Find columns that exist in both tables
common_cols = list(set(stg_df.columns) & set(final_df.columns))

if common_cols:
    print(f"  Comparing {len(common_cols)} columns: {', '.join(sorted(common_cols))}")
    
    # Sort both tables the same way for comparison
    stg_sorted = stg_df[common_cols].sort_values(by=common_cols).reset_index(drop=True)
    final_sorted = final_df[common_cols].sort_values(by=common_cols).reset_index(drop=True)
    
    # Check if they're identical
    if stg_sorted.equals(final_sorted):
        print(f"  Result: PASS - All common data matches")
    else:
        print(f"  Result: FAIL - Data mismatch found")
        
        # Show where differences are
        diff_rows = ~(stg_sorted == final_sorted).all(axis=1)
        diff_count = diff_rows.sum()
        print(f"\n  {diff_count} rows have differences (showing first 3):")
        print("\n  Staging:")
        print(stg_sorted[diff_rows].head(3).to_string())
        print("\n  Final:")
        print(final_sorted[diff_rows].head(3).to_string())
else:
    print("  WARNING: No common columns found")

print("\n" + "="*60)
print("END OF REPORT")
print("="*60)

C:\Users\anusr\AppData\Local\Temp\ipykernel_14668\3287352143.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stg_df = pd.read_sql("SELECT * FROM SALESRECORD;", conn)
C:\Users\anusr\AppData\Local\Temp\ipykernel_14668\3287352143.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  final_df = pd.read_sql("SELECT * FROM SALESRECORDFINAL;", conn)


DATA VALIDATION REPORT

TEST 1: Row Count
  Staging: 524 rows
  Final:   524 rows
  Result:  PASS

TEST 2: Null Value Check
  Staging Table:
    ORDER_DATE: 0 nulls [PASS]
    DATETIME: 0 nulls [PASS]
    MONEY: 0 nulls [PASS]
    CASH_TYPE: 0 nulls [PASS]
    COFFEE_NAME: 0 nulls [PASS]

  Final Table:
    ORDER_DATE: 0 nulls [PASS]
    DATETIME: 0 nulls [PASS]
    MONEY: 0 nulls [PASS]
    CASH_TYPE: 0 nulls [PASS]
    COFFEE_NAME: 0 nulls [PASS]
    TOTAL: 0 nulls [PASS]

TEST 3: Business Logic Validation
  Formula: TOTAL = MONEY * 1.05
  Errors: 0
  Result: PASS

TEST 4: Data Consistency Check
  Comparing 5 columns: CASH_TYPE, COFFEE_NAME, DATETIME, MONEY, ORDER_DATE
  Result: PASS - All common data matches

END OF REPORT
